In [ ]:
import numpy as np
import pandas as pd


In [ ]:
df =  pd.read_csv('movies.csv')

In [ ]:
df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
10324,146684,Cosmic Scrat-tastrophe (2015),Animation|Children|Comedy
10325,146878,Le Grand Restaurant (1966),Comedy
10326,148238,A Very Murray Christmas (2015),Comedy
10327,148626,The Big Short (2015),Drama


In [ ]:
df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10329 entries, 0 to 10328
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  10329 non-null  int64 
 1   title    10329 non-null  object
 2   genres   10329 non-null  object
dtypes: int64(1), object(2)
memory usage: 242.2+ KB


In [ ]:
df.describe()

,movieId
count,10329.000000
mean,31924.282893
std,37734.741149
min,1.000000
25%,3240.000000
50%,7088.000000
75%,59900.000000
max,149532.000000


In [ ]:
df.shape

(10329, 3)

In [ ]:
movies = df['movieId'].unique().tolist()
len(movies)

10329

In [ ]:
df.isnull().any()

movieId    False
title      False
genres     False
dtype: bool

In [ ]:
df.duplicated().any()

False

In [ ]:
vectorizer = CountVectorizer(tokenizer=lambda x: x.split('|'))
genre_matrix = vectorizer.fit_transform(data['genres']).toarray()

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
genre_similarity = cosine_similarity(genre_matrix)

In [ ]:
movie_indices = data['movieId'].tolist()


In [ ]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
def recommend_movies_based_on_genres(movie_id, num_recommendations=5):
    movie_index = movie_indices.index(movie_id)
    similar_movies = genre_similarity[movie_index]
    recommended_movie_indices = similar_movies.argsort()[-num_recommendations-1:-1][::-1]
    recommended_movie_ids = [movie_indices[idx] for idx in recommended_movie_indices]
    recommended_movies = data[data['movieId'].isin(recommended_movie_ids)]['title'].tolist()
    return recommended_movies

In [ ]:
mae_scores = []
rmse_scores = []

for _, row in test_data.iterrows():
    movie_id = row['movieId']
    actual_genres = row['genres']
    recommended_movies = recommend_movies_based_on_genres(movie_id)

    actual_genres_set = set(actual_genres.split('|'))
    recommended_genres_set = set('|'.join(data[data['title'].isin(recommended_movies)]['genres']).split('|'))

    mae_scores.append(len(actual_genres_set - recommended_genres_set))
    rmse_scores.append((len(actual_genres_set - recommended_genres_set)) ** 2)

mean_mae = sum(mae_scores) / len(mae_scores)
rmse = (sum(rmse_scores) / len(rmse_scores)) ** 0.5

print(f"Mean Absolute Error: {mean_mae}")
print(f"Root Mean Squared Error: {rmse}")

Mean Absolute Error: 0.007744433688286544
Root Mean Squared Error: 0.08800246410349283


In [ ]:
def recommend_movies_for_user(user_preferences, num_recommendations=10):
    user_preferences_set = set(user_preferences.split('|'))

    user_genre_vector = vectorizer.transform(['|'.join(user_preferences_set)]).toarray()
    genre_similarity_with_user = cosine_similarity(genre_matrix, user_genre_vector)

    similar_movie_indices = genre_similarity_with_user.argsort(axis=0)[-num_recommendations:][::-1].flatten()
    recommended_movie_ids = [movie_indices[idx] for idx in similar_movie_indices]
    recommended_movies = data[data['movieId'].isin(recommended_movie_ids)]['title'].tolist()
    return recommended_movies

In [ ]:
newuser_preferences = 'Horror|Children'
recommended_movies = recommend_movies_for_user(newuser_preferences)

print("\nRecommended movies:- \n")

for movie in recommended_movies:
    print(movie)



Recommended movies:- 

Lord of Illusions (1995)
Cemetery Man (Dellamorte Dellamore) (1994)
Blood for Dracula (Andy Warhol's Dracula) (1974)
Gate, The (1987)
976-EVIL (1989)
Return of the Vampire, The (1944)
Wolfen (1981)
Satanic Rites of Dracula, The (1974)
Day of the Dead (2008)
It Follows (2014)
